<h2>Import packages and install histomics_detect</h2>

In [1]:
import sys
import tensorflow as tf

#install histomics_detect
!pip install -e /tf/notebooks/histomics_detect

#add to system path
sys.path.append('/tf/notebooks/histomics_detect/')

Obtaining file:///tf/notebooks/histomics_detect
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 679 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 36.2 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 36.7 MB/s eta 0:00:01
  Running setup.py develop for histomics-detect
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


<h2>Define dataset parameters and create datasets</h2>

In [2]:
#import dataset related packages
from histomics_detect.io import roi_tensors
from histomics_detect.io import read_roi
from histomics_detect.augmentation import crop, flip, jitter, shrink
import numpy as np
import os
from PIL import Image

#input data path
path = '/tf/notebooks/DCC/data/'

#training parameters
train_tile = 224 #input image size
min_area_thresh = 0.5 # % of object area that must be in crop to be included

#supply a function for extracting sample id from filename
def parse_filename(file):
    return os.path.split(file)[1].split('.')[2]

#build (.png, image size, slide, .csv) file tuples
files = os.listdir(path)
pngs = [path + f for f in files if os.path.splitext(f)[1] == '.png']
files = [(png, Image.open(png).size, parse_filename(png),
          '.'.join(png.split('.')[0:-1]) + '.csv') for png in pngs]

#filter on png size
files = [(png, size, slide, csv) for (png, size, slide, csv) in files
         if (size[0] > train_tile) and (size[1] > train_tile)]

#randomly assign 20% of slides to validation
slides = list(set([file[2] for file in files]))

id = np.random.randint(0, len(slides)-1, size=(np.ceil(0.2*len(slides)).astype(np.int32)))
validation = [slide for (i, slide) in enumerate(slides) if i in id]
training = list(set(slides).difference(validation))
training_files = [(png, csv) for (png, size, slide, csv) in files if slide in training]
validation_files = [(png, csv) for (png, size, slide, csv) in files if slide in validation]

#convert to tensors
training_rois = roi_tensors(training_files)
validation_rois = roi_tensors(validation_files)

#arguments
width = tf.constant(train_tile, tf.int32)
height = tf.constant(train_tile, tf.int32)
min_area = tf.constant(min_area_thresh, tf.float32)

#build training dataset
ds_train_roi = tf.data.Dataset.from_tensor_slices(training_rois)
ds_train_roi = ds_train_roi.map(lambda x: read_roi(x))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*crop(x, y, width, height,
                                                       min_area_thresh), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*flip(x, y), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, jitter(y, 0.05), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, shrink(y, 0.05), z))
ds_train_roi = ds_train_roi.prefetch(tf.data.experimental.AUTOTUNE)

#build validation datasets
ds_validation_roi = tf.data.Dataset.from_tensor_slices(validation_rois)
ds_validation_roi = ds_validation_roi.map(lambda x: read_roi(x))
ds_validation_roi = ds_validation_roi.prefetch(tf.data.experimental.AUTOTUNE)

<h2>Create and train detection model</h2>

In [4]:
#import network generation and training packages
from histomics_detect.networks.rpns import rpn
from histomics_detect.models.faster_rcnn import FasterRCNN
from histomics_detect.models.faster_rcnn import CustomCallback

#choices for anchor sizes - all anchors 1:1 aspect ratio
anchor_px = tf.constant([32, 64, 96], dtype=tf.int32) #width/height of square anchors in pixels at input mag.

#feature network parameters
backbone_stride = 1 #strides in feature generation network convolution
backbone_blocks = 14
backbone_dimension = 256 #number of features generated by rpn convolution

#rpn network parameters
rpn_kernel = [3] #kernel size for rpn convolution
rpn_act_conv = ['relu'] #activation for rpn convolutional layers

#anchor filtering parameters
neg_max = 128 #maximum number of negative/positive anchors to keep in each roi
pos_max = 128
rpn_lmbda = 10.0 #weighting for rpn regression loss
roialign_tiles = 3.0 #roialign - number of horizontal/vertical tiles in a proposal
roialing_pool = 2.0 #roialign - number of horizontal/vertical samples in each tile

#create backbone and rpn networks
resnet50 = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(train_tile, train_tile, 3), pooling=None)
rpnetwork, backbone = rpn(resnet50, n_anchors=tf.size(anchor_px),
                          stride=backbone_stride, blocks=backbone_blocks, 
                          kernels=rpn_kernel, dimensions=[backbone_dimension],
                          activations=rpn_act_conv)

#create FasterRCNN keras model
model = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)

#compile FasterRCNN model with losses
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=[tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    tf.keras.losses.Huber()])

#fit FasterRCNN model
model.fit(x=ds_train_roi, batch_size=1, epochs=50, verbose=1, callbacks=[CustomCallback()],
          validation_data=ds_validation_roi, validation_freq=50)

Epoch 1/50
136/136 [==============================] - 18s 59ms/step - rpn_objectness: 0.7073 - rpn_regression: 0.1927 - align_regression: 1.1644 - mean_iou_rpn: 0.6171 - mean_iou_align: -inf - auc_roc: 0.6714 - pr_roc: 0.4923 - true_positives_1: 421.0000 - false_negatives_1: 1039.0000 - false_positives_1: 332.0000
Epoch 2/50
136/136 [==============================] - 8s 60ms/step - rpn_objectness: 0.6666 - rpn_regression: 0.0243 - align_regression: 0.0312 - mean_iou_rpn: 0.7595 - mean_iou_align: 0.6965 - auc_roc: 0.8500 - pr_roc: 0.7359 - true_positives_1: 367.0000 - false_negatives_1: 1117.0000 - false_positives_1: 46.0000
Epoch 3/50
136/136 [==============================] - 8s 60ms/step - rpn_objectness: 0.6410 - rpn_regression: 0.0202 - align_regression: 0.0227 - mean_iou_rpn: 0.7658 - mean_iou_align: 0.7624 - auc_roc: 0.9010 - pr_roc: 0.8157 - true_positives_1: 455.0000 - false_negatives_1: 987.0000 - false_positives_1: 38.0000
Epoch 4/50
136/136 [==============================] -

In [5]:
#inference on a single image
data = list(ds_validation_roi.shuffle(100).take(1).as_numpy_iterator())[0]
rgb = data[0]
call_output = model(rgb)

#inference on multiple images from a tf.data.Dataset - merges outputs
predict_output = model.predict(ds_validation_roi)

#performance evaluation on multiple images from a tf.data.Dataset
metrics = model.evaluate(ds_validation_roi)

rpn precision:  0.83495145631067957
rpn recall:  0.9555555555555556
rpn tp:  86
rpn fp:  17
rpn fn:  4
align precision:  0.84466019417475724
align recall:  0.96666666666666667
align tp:  87
align fp:  16
align fn:  3
 1/33 [..............................] - ETA: 40s - mean_iou_rpn: 0.0085 - mean_iou_align: 0.0088 - auc_roc: 1.0000 - pr_roc: 1.0000 - true_positives_1: 103.0000 - false_negatives_1: 0.0000e+00 - false_positives_1: 0.0000e+00rpn precision:  0.76190476190476186
rpn recall:  0.62745098039215685
rpn tp:  32
rpn fp:  10
rpn fn:  19
align precision:  0.7857142857142857
align recall:  0.6470588235294118
align tp:  33
align fp:  9
align fn:  18
 2/33 [>.............................] - ETA: 11s - mean_iou_rpn: 0.0097 - mean_iou_align: 0.0100 - auc_roc: 1.0000 - pr_roc: 1.0000 - true_positives_1: 145.0000 - false_negatives_1: 0.0000e+00 - false_positives_1: 0.0000e+00rpn precision:  0.95238095238095233
rpn recall:  0.95238095238095233
rpn tp:  40
rpn fp:  2
rpn fn:  2
align precisi

<h2>Save and Load Model Weights</h2>

In [ ]:
weight_path = "cpk"
model.save_weights(weight_path)
model_reload = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)
model_reload.load_weights(weight_path)

align_reg = model.fastrcnn(interpolated)
align_reg_reload = model_reload.fastrcnn(interpolated)

assert tf.reduce_sum(tf.cast(align_reg_reload == align_reg, tf.int32)) == tf.math.reduce_prod(tf.shape(align_reg))